# COMET InSAR Workshop 2024: LiCSBAS Tutorial

The aim of this tutorial is to understand and complete the processing steps of open-source LiCSBAS software required to prepare a time series analysis for Campi Flegrei using LiCSAR data from 2016-2021.  


This notebook is provided with default parameters using a subset of data to ensure that the practical session can be completed in the allotted time. The practical instructor will talk through the parameters and the operation of the notebook; however, you are not required to make edits for the notebook to run.

### please run the following cell first to load some necessary modules:  
(to run a cell, just click into it and press shift-Enter)  

#### Note: in case your notebook gets frozen/kernel disconnected, try clicking Kernel->Restart (rather than Refreshing page). If this runs ok, you may just rerun the following cell, and then skip back to the cell you were in before.

**cell \[ 00 \]** - this is a naming of a cell below:

In [ ]:
from IPython.display import Image
import os

# below is for the local jupyhub env - should not affect the binder version
os.environ['PATH'] = os.environ['PATH']+':/data/notebooks/repositories/LiCSBAS/bin'
import sys
sys.path.append('/data/notebooks/repositories/LiCSBAS/LiCSBAS_lib')

import warnings
warnings.filterwarnings('ignore')

# some extra functions
def display(png):
    if not os.path.exists(png):
        print('This png file does not exist, recheck please')
    else:
        return Image(png)

def printbold(instr='done'):
    print('\033[1m' + instr + '\033[0m')

# selected frame
frame = '022D_04826_121209' # for tutorial

# Once you identified the frame ID to process and confirmed that 
# the sufficient interferograms are available on the frame, 
# make a processing directory and change to the directory
framedir = os.path.join(os.environ['HOME'],frame)
try:
    os.mkdir(framedir)
except:
    pass

os.chdir(framedir)

# below check is for the local jupyhub env only, the files should have already been extracted if you run through binder
if not os.path.exists('GEOCml2GACOS'):
    print('extracting licsbas data')
    os.system('7za x /data/notebooks/COMET_InSAR_Workshop_2024/Day2_LiCSBAS/tutorial.7z')
    # or this one - a result of time series only:
    os.system('7za x /data/notebooks/COMET_InSAR_Workshop_2024/Day2_LiCSBAS/tutorialts.7z')


print('prerequisites loaded, thank you. Running in directory:')
print(os.getcwd())

printbold()

# LiCSBAS Tutorial

LiCSBAS is an open-source package in Python and bash to carry out InSAR time series analysis using LiCSAR interfeometric products (primarily unwrapped interferograms and coherence) which are freely available on the [COMET-LiCS web portal](https://comet.nerc.ac.uk/COMET-LiCS-portal/).



Users can easily and automatically derive the time series and velocity of the displacement if sufficient LiCSAR products are available in the area of interest. LiCSBAS also contains visualization tools to interactively display the time series of displacement to help investigation and interpretation of the results. Current release offers various improvements including option to perform customized unwrapping procedure using *LiCSBAS02to05_unwrap.py*.  

## Documentation

The [COMET LiCSBAS repository](https://github.com/comet-licsar/licsbas) is an actively developed fork of the [original LiCSBAS](https://github.com/yumorishita/LiCSBAS) tool developed by Dr. Yu Morishita during his stay at the University of Leeds. For more detailed information, see the [**wiki**](https://github.com/yumorishita/LiCSBAS/wiki) pages and [quick start](https://github.com/yumorishita/LiCSBAS/wiki/2_0_workflow#quick-start).  

This tutorial is run through Jupyter Notebook - see short [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial) if you are not familiar with this environment.

## Standard LiCSBAS Workflow

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/workflow.png" width=650>

**Note:** Current workflow is extended by several optional processes, such as advanced selection of reference point (LiCSBAS120 by Dr. Qi Ou), pixel-based nullification of unwrapping errors (part of LiCSBAS12, by Dr. Lin Shen), advanced unwrapping procedure (LiCSBAS02to05, by Dr. Milan Lazecky), estimation of non-linear model and offsets (LiCSBAS_cum2vel, by Dr. Yu Morishita and M. Lazecky), ability to load ALOS interferograms (LiCSBAS01_get_geotiff_ALOS included from Dr. Yu Morishita's updated solutions), among various other changes. As we keep using standard LiCSBAS functionality within this tutorial, we did not update flowchart and other figures and leave them for investigation of individual parameters of the LiCSBAS scripts.

# Module 0: Preparing the dataset

This tutorial will run on a preprocessed dataset (i.e. modules 0 are complete here) that is clipped in both time and space, and cover covers volcanic area of Campi Flegrei in Napoli region (Italy). The figure below shows GPS network measurements over the Napoli region, published in https://www.mdpi.com/2072-4292/13/14/2725. The Campi Flegrei caldera (in the middle) has been inflating since 2010s, a continuous uplift can be observed from the GPS station time series (middle plot).  

<img src="tutorial/campi_flegrei_gps.png" width="1000">

You may use the following module 0 steps to process another frame of your interest. **For the purposes of this tutorial, they are marked as skipped**.  


Note: you may identify LiCSAR frames covering your own area of interest (AOI) in your free time later. For this, you may just check the [COMET LiCSAR portal](https://comet.nerc.ac.uk/COMET-LiCS-portal/).  


## (skipping) Step 0-1: Download GeoTIFF from COMET-LiCS web portal


### LiCSBAS01_get_geotiff.py
This script downloads GeoTIFF files of unw (unwrapped interferogram) and cc (coherence) in the specified frame ID from COMET-LiCS web portal.  
The -f option is not necessary when the frame ID can be automatically identified from the name of the working directory.  
GACOS data can also be downloaded if available, as well as phase data (and intensity) required for the lately added unwrapping procedure.  
Existing GeoTIFF files are not re-downloaded to save time, i.e., only the newly available data will be downloaded.  

**Note:** LiCSBAS is capable of running on any network of standard interferograms if these exist in GEOC/YYYYMMDD_YYYYMMDD together with coherence (either in 0-1 or 0-255 range)

**cell \[ 01 \]**

In [ ]:
! LiCSBAS01_get_geotiff.py -h
#e.g. LiCSBAS01_get_geotiff.py -f 167D_04884_131212 --get_gacos --n_para 1
#printbold()

## (skipping) Step 0-2: Convert and downsample

### LiCSBAS02_ml_prep.py
This script converts GeoTIFF files of unw and cc to float32 and uint8 format, respectively, for further time series analysis, and also downsamples (multilooks) data if specified.  
Existing files are not re-created to save time, i.e., only the newly available data will be processed.  

**Note:** Use of LiCSBAS02to05_unwrap.py replaces this and following 0-x optional steps, however they still can be applied to its output.

**cell \[ 02 \]**

In [ ]:
! LiCSBAS02_ml_prep.py -h
# e.g. LiCSBAS02_ml_prep.py -i GEOC -o GEOCml10 -n 10 --npara 1
printbold()

# (skipping) Step 0-3: GACOS correction (optional)

### LiCSBAS03op_GACOS.py
This script applies a tropospheric correction to unw data using GACOS data.  
GACOS data may be automatically downloaded from COMET-LiCS web at step01 (if available), or could be externally obtained by requesting on a [GACOS web](http://www.gacos.net/).  
(if you request the GACOS data through the GACOS web, the dates and time of interest can be found in baselines and slc.mli.par, respectively. Note that original GACOS data comes as zenith total delay, ZTD. This is automatically converted to the slant direction of the satellite, i.e. into slant total delay, SLTD).  

 
The impact of the correction can be visually checked by showing GACOS_info.png and \*/\*.gacos.png:  
- unw_org - original unwrapped data (STD = overall standard deviation of all unw values)
- unw_cor - unwrapped data after GACOS correction
- dsltd - differential (respective to the interferometric pair) correction image from GACOS ZTD data converted to SLTD (see above)

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20170407_20170501.gacos.png" width=600>  

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/GACOS_info.png" width=600>

**cell \[ 03 \]**

In [ ]:
! LiCSBAS03op_GACOS.py -h
# e.g. LiCSBAS03op_GACOS.py -i GEOCml10 -o GEOCml10GACOS --fillhole --n_para 1
printbold()

## (skipping) Step 0-4: Mask UNW (optional)

### LiCSBAS04op_mask_unw.py
This script masks specified areas or low coherence areas in the unw data.  
The masking is effective when the unw data include areas which have many unwrapping errors and are not of interest, and can improve the result of Step 1-2 (loop closure).   

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20150524_20150617.unw_mask.png" width=600>

**Note:** In practice, an option to mask data based on their individual coherence (parameter *-s*) is valuable for reliable inversion in the later steps.

**cell \[ 04 \]**

In [ ]:
! LiCSBAS04op_mask_unw.py -h
printbold()

## (skipping) Step 0-5: Clip UNW (optional)

### LiCSBAS05op_clip_unw.py
This script clips a specified rectangular area of interest from unw and cc data.  
The clipping can make the data size smaller and processing faster, and improve the quality measures calculated for the whole scene, such as avg loop phase closure result of Step 1-2.  

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20150524_20150617.unw_clip.png" width=400>

**cell \[ 05 \]**

In [ ]:
! LiCSBAS05op_clip_unw.py -h
# e.g. LiCSBAS05op_clip_unw.py -i GEOCml10GACOS -o GEOCml10GACOSclip -g 14.03/14.22/40.78/40.90 --n_para 1
printbold()

# Running steps 11-16 using existing preprocessed data

LiCSBAS processing directory has following folders:
- GEOC - location for the original data, (auto-)downloaded from COMET LiCSAR
- GEOCmlX - result of Step 0-2 - X-times downsampled converted data
- GEOCmlXGACOS - optional - result of GACOS correction (steps 0-4 and 0-5 also generate optional 'final pre-processed dataset'
- GEOCmlX\*clip - optional - spatially clipped dataset
- TS_GEOC* - folder containing outputs of time series processing (see further)

**Note:** We have preprocessed data for the given frame for you, for the purposes of this tutorial.

**cell \[ 06 \]**

In [ ]:
# check the contents of the folder
! pwd
! ls


In [ ]:
# check the number of interferograms
! echo The folder contains `ls GEOCml2GACOSclip | wc -l` interferograms
! echo "last 30 folders+files:"
! ls GEOCml2GACOSclip | tail -n 30

**cell \[ 07 \]**

In [ ]:
# check the interferograms network:
! ls GEOCml2GACOSclip | grep '_20' > ifg_list.txt
print('generating network plot')
! LiCSBAS_plot_network.py -i ifg_list.txt -b GEOCml2GACOSclip/baselines -o network.png >/dev/null 2>/dev/null
print('first 10 interferogram pairs:')
! head -n 10 ifg_list.txt
display('network.png')

**cell \[ 08 \]**

In [ ]:
# look into some random interferogram:
pair = '20161014_20161026'
png = os.path.join('GEOCml2GACOSclip',pair,pair+'.unw.png')
display(png)

**cell \[ 09 \]**

In [ ]:
# see effect of GACOS of this selected interferogram:
# note GACOS corrections are created before clipping, thus they are in the second extracted folder
pair='20161008_20161026'
#pair='20161002_20161020'  # pair causing higher std after GACOS
print('unw_org - original unwrapped data')
print('unw_cor - corrected unw data')
print('dsltd - differential slant direction total delay estimate from GACOS data')
print('colour scale: 6pi/colour cycle')
png = os.path.join('GEOCml2GACOS',pair,pair+'.gacos.png')
display(png)

In [ ]:
# look into some interferogram - e.g. see effect of GACOS:
! head -n 10 GEOCml2GACOSclip/GACOS_info.txt
png = 'GEOCml2GACOSclip/GACOS_info.png'
display(png)

# Performing basic time series analysis

# Step 1-1: Quality Check

### LiCSBAS11_check_unw.py
This script checks quality of unw data and identifies bad interferograms based on average coherence and coverage of the unw data.  
This also prepares a time series working directory (overwrite if already exists).  

Examples of low quality (to be removed) interferograms:  
<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20180108_20180213.unw.geo.png" width="300">
<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20150325_20150418.unw.geo.png" width="300">  

**Note:** LiCSAR team works on updated quality checking together to flag and remove interferograms containing coregistration or other errors. An optional parameter **-s** would detect interferograms with azimuth ramp higher than approx. 0.1 mm/km, as often result due to strong ionospheric influence.  
<img src="tutorial/licsbas11_s.png" width=800>

**cells \[ 10 \]**

In [ ]:
! LiCSBAS11_check_unw.py -h
# e.g. LiCSBAS11_check_unw.py -d GEOCml2GACOSclip
printbold()

In [ ]:
print('running the command')
! LiCSBAS11_check_unw.py -d GEOCml2GACOSclip
printbold()
# ETA few seconds

**cells \[ 11 \]**

In [ ]:
# check outputs of the step:
! ls TS_GEOCml2GACOSclip
# outputs of step 1-1 are in folders starting by 11

In [ ]:
# see bad ifgs:
! ls TS_GEOCml2GACOSclip/11bad_ifg_ras

In [ ]:
# check stats - perhaps change thresholds?
! head -n 30 TS_GEOCml2GACOSclip/info/11ifg_stats.txt

In [ ]:
# see one of the discarded ifgs:
pair='20200414_20200420'
png = os.path.join('TS_GEOCml2GACOSclip','11bad_ifg_ras',pair+'.unw.png')
#png = os.path.join('GEOCml2GACOSclip',pair,pair+'.unw.png')
display(png)

In [ ]:
#check network
png='TS_GEOCml2GACOSclip/network/network11.png'
display(png)

# Step 1-2: Loop Closure

### LiCSBAS12_loop_closure.py
This script identifies bad unw by checking loop closures of interferometric triplets, i.e. non-zero residuals 
$$\phi_{123}=\phi_{12}+\phi_{23}-\phi_{13}$$


<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20171203_20171215_20171227_loop.png" width="400">

This procedure runs 4 iterations where the following is applied, to investigate loop closure: 
1. iteration - identify loops and mark/not use unw ifgs of higher overall RMSE than given threshold, tolerating $2\pi$ phase jumps
2. iteration - check loop errors per pixel and select stable reference point (all valid unw data and a minimum of loop phase RMSE)
3. iteration - use reference area, do not tolerate $2\pi$ jumps, re-identify unw ifgs of higher overall RMSE than given threshold and remove them as bad
4. iteration - calculate statistics (n_loop_err, n_ifg_noloop, and their ratio) and optionally nullify pixels of unws that only appear in non-closed loop -> good pixels of bad ifgs are still used  


**cells \[ 12 \]**

In [ ]:
! LiCSBAS12_loop_closure.py -h
# e.g. LiCSBAS12_loop_closure.py -d GEOCml10GACOS -l 3 --n_para 1
printbold()

In [ ]:
print('running the command')
! LiCSBAS12_loop_closure.py -d GEOCml2GACOSclip -l 1.5 --n_para 1 --multi_prime
printbold()
# ETA 7 min., rerunning: ETA few seconds

In [ ]:
import LiCSBAS_io_lib as io_lib
import LiCSBAS_loop_lib as loop_lib
import LiCSBAS_tools_lib as tools_lib
import LiCSBAS_inv_lib as inv_lib
import LiCSBAS_plot_lib as plot_lib
import xarray as xr
import cmcrameri.cm as cmc
import matplotlib.pyplot as plt

In [ ]:
tools_lib.get_cmap('cmc.romaO')

In [ ]:
! grep import /data/notebooks/repositories/LiCSBAS/bin/LiCSBAS12_loop_closure.py

**cells \[ 13 \]**

In [ ]:
! head -n 10 TS_GEOCml2GACOSclip/12loop/loop_info.txt
! echo "..."
! tail -n 30 TS_GEOCml2GACOSclip/12loop/loop_info.txt

In [ ]:
! ls TS_GEOCml2GACOSclip/12loop/bad_loop_png

In [ ]:
# see one of the triplets of discarded ifgs:
triplet = '20171226_20180101_20180113'
#triplet = '20171108 20171226 20180101' # example of bad loop
#triplet = '20161014 20161020 20161101' # example of candidate to bad loop that is used further
triplet = triplet.replace(' ','_')
png = os.path.join('TS_GEOCml2GACOSclip','12loop','bad_loop_png',triplet+'_loop.png')
if not os.path.exists(png):
    png = os.path.join('TS_GEOCml2GACOSclip','12loop','good_loop_png',triplet+'_loop.png')
print('Note: the figure below was generated during 1st iteration, i.e. before referencing/without multi-prime')
display(png)

In [ ]:
# check also no loop ifgs:
! echo "There are "`ls TS_GEOCml2GACOSclip/12no_loop_ifg_ras | wc -l`" pairs identified as not forming a complete loop triplet"
! ls TS_GEOCml2GACOSclip/12no_loop_ifg_ras

In [ ]:
#check network
png='TS_GEOCml2GACOSclip/network/network12.png'
display(png)

# Step 1-3: Small Baseline Inversion

### LiCSBAS13_sb_inv.py
This script inverts the SB network of unw to obtain the time series and velocity using NSBAS ([López-Quiroz et al., 2009](https://www.sciencedirect.com/science/article/pii/S0926985109000251); [Doin et al., 2011](https://earth.esa.int/documents/10174/1573056/Presentation_small_baseline_NSBAS_Etna_deformation_Envisat.pdf)) approach.  
A stable reference point is determined after the inversion.  
RMS of the time series wrt median among all points is calculated for each point.  
Then the point with minimum RMS and minimum n_gap is selected as new stable reference point.

**cells \[ 14 \]**

In [ ]:
! LiCSBAS13_sb_inv.py -h 
# e.g. LiCSBAS13_sb_inv.py -d GEOCml10GACOS --n_para 1 --mem_size 1000
printbold()

In [ ]:
print('running the command')
! LiCSBAS13_sb_inv.py -d GEOCml2GACOSclip --n_para 1 --mem_size 1000 --singular --nopngs #--only_sb
#without --singular:  (original constrained SBAS method aka NSBAS)
#ETA 25 min. ... although, on your local computer this would take approx. 5 minutes
#
#with --singular:
#ETA 7 minutes, but separate subsets can be connected in lower precision
#
#with --only_sb
#ETA 1 minute, but would skip all points that contain some NaN (only pixels with full SBAS network are inverted)

printbold()

LiCSBAS procedure will first invert matrix for points having all unwrapped values (no NaNs) using the full NSBAS set of equations (same result as for $\bf d = \bf G \bf m$ where $\bf d$ is set of unwrapped data related to $\bf m$ that is a vector of incremental displacements, through design matrix $\bf G$) and then runs for incomplete points to find $\bf m$, filling empty values under assumption that the displacement is linear in time, i.e. solving points with missing values using set of equations:
$$\bf d = \bf G \bf m$$ $$0 = 0.0001(B m - vt - c)$$

for details (e.g. definition of matrix B) see https://doi.org/10.3390/RS12030424  

In our case, using **--singular** approach, we simplify the calculation of the gaps (missing values) by using only the partial range of SBAS-extracted cumulated increments, and filling the gaps assuming constant linear velocity connecting epochs between each gap. Although this leads to similar outputs, the computation is significantly simpler and keeps non-linear information better as connected clusters are not aligned along the estimated linear trend. Yet it is considered more prone to mistakes due to errors at the gap connecting epochs. In future release this can be replaced by extrapolation using Gaussian window in time.


**cells \[ 15 \]**

In [ ]:
#check network
png='TS_GEOCml2GACOSclip/network/network13.png'
display(png)

In [ ]:
! ls TS*/results/*.png
png='TS_GEOCml2GACOSclip/results/vel.png'
display(png)

# Step 1-4: Calculate STD of Velocity by Bootstrap

### LiCSBAS14_vel_std.py
This script primarily calculates the standard deviation of the velocity by the bootstrap method and STC (spatio-temporal consistency; [Hanssen et al., 2008](https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/documents/Hanssen_2008.pdf)) but can also be used to optionally re-estimate velocity with outlier detection/removal.

**cells \[ 16 \]**

In [ ]:
! LiCSBAS14_vel_std.py -h
# e.g. LiCSBAS14_vel_std.py -t TS_GEOCml10GACOS
printbold()

In [ ]:
print('running the command')
! LiCSBAS14_vel_std.py -t TS_GEOCml2GACOSclip --mem_size 1000
# ETA 5 sec
printbold()

**cells \[ 17 \]**

In [ ]:
! ls TS*/results/*.png -alh
png='TS_GEOCml2GACOSclip/results/vstd.png'
display(png)

In [ ]:
png='TS_GEOCml2GACOSclip/results/stc.png'
display(png)

# Step 1-5: Mask Time Series

### LiCSBAS15_mask_ts.py

This script makes a mask for time series using several noise indices.  
The pixel is masked if any of the values of the noise indices for a pixel is worse (larger or smaller) than a specified threshold.  

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/mask_ts.png" width="900">

| Noise index | Meaning |
| --- | ----------- |
| coh_avg | Average value of the interferometric coherence across the stack (0–1) |
| n_unw | Number of unwrapped data used in the time series calculation (as a ratio to the number of images; i.e., if 1.5, n_unw=1.5*n_im) |
| vstd | Standard deviation of the velocity (mm/yr) estimated in Step 1-4 |
| maxTlen | Maximum time length of the connected network (years). The larger the value, the better the precision in the velocity estimate. |
| n_gap | Number of gaps in the interferogram network and time series breaks. |
| **n_gap_merged** | (only if set as parameter *--n_gap_use_merged*) Modified n_gap counting neighbouring gaps as one. |
| stc | Spatio-temporal consistency (mm) (Hanssen et al., 2008), estimated in Step 1-4. |
| n_ifg_noloop | Number of interferograms with no loops that cannot be checked by the loop closure and possibly have unidentified unwrapping errors. |
| n_loop_err | Number of the unclosed loops after the network refinement in Step 1-2. |
| **n_loop_err_rat** | Ratio of n_loop_err per number of checked loops. Will replace n_loop_err in the next major release. |
| **loop_ph_avg_abs** | Average of the absolute wrapped phase loop closure (aka phase bias) (rad). |
| resid_rms | RMS of residuals in the small baseline (SB) inversion (mm). |

(bold: newly added signatures since the last release)

**cells \[ 18 \]**

In [ ]:
! LiCSBAS15_mask_ts.py -h
# e.g. LiCSBAS15_mask_ts.py -t TS_GEOCml10GACOS -c 0.05 -u 1.5 -v 100 -T 1 -g 10 -s 5  -i 50 -l 5 -r 2
printbold()

In [ ]:
print('running the command')
! LiCSBAS15_mask_ts.py -t TS_GEOCml2GACOSclip 
printbold()
png = 'TS_GEOCml2GACOSclip/results/vel.mskd.png'
display(png)

In [ ]:
print('Remind Step 1-2 - there were 64 interferograms not completing a loop! thus n_ifg_noloop=64 ')
png = 'TS_GEOCml2GACOSclip/mask_ts.png'
display(png)

In [ ]:
print('running the command')
! LiCSBAS15_mask_ts.py -t TS_GEOCml2GACOSclip -i 70 -l 10 -r 2
# ETA 2 sec
printbold()
png = 'TS_GEOCml2GACOSclip/results/vel.mskd.png'
display(png)

**cell \[ 19 \]**

In [ ]:
png = 'TS_GEOCml2GACOSclip/mask_ts.png'
display(png)

# Step 1-6: Filter (& Deramp) Time Series

### LiCSBAS16_filt_ts.py
This script applies spatio-temporal filter (HP in time and LP in space with gaussian kernel, same as StaMPS) to the time series of displacement.  
Deramping (1D, bilinear, or 2D polynomial) can also be applied if -r option is used.  
Topography-correlated components (linear with elevation) can also be subtracted with --hgt_linear option simultaneously with deramping before spatio-temporal filtering.  
Further on, we incorporated DEM error estimation implemented in [LiCSBAS2](https://github.com/yumorishita/LiCSBAS2).
The impact of filtering (deramp and linear elevation as well) can be visually checked by showing 16filt\*/\*png.  
A stable reference point is determined after the filtering in the same manner as in Step 1-3.  

### Previews:
Deramping:  
<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20180102_deramp.png" width="600">  

Removing phase correlated to topography:  
<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20180102_hgt_linear.png" width="600">  
<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20180102_hgt_corr.png" width="400">  

Spatio-temporal filtering:  
<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/20180102_filt.png" width="600">

**cells \[ 20 \]**

In [ ]:
! LiCSBAS16_filt_ts.py -h
# e.g. LiCSBAS16_filt_ts.py -t TS_GEOCml2GACOSclip -r 2 --n_para 1
printbold()

In [ ]:
print('running the command')
! LiCSBAS16_filt_ts.py -t TS_GEOCml2GACOSclip -r 1 --n_para 1 --hgt_linear --nofilter
# ETA (on typical local computer): 10 min. with hgt_linear, 7 minutes without
# Binder: >20 minutes if using spatio-temporal filter -> here we only deramp and use hgt_linear
printbold()

**cells \[ 21 \]**

In [ ]:
print('first 30 files of filtered dataset')
! ls TS_GEOCml2GACOSclip/16filt_cum | head -n 30

In [ ]:
#epoch = '20161026'
epoch = '20161008'

In [ ]:
print('effect of deramping:')
png='TS_GEOCml2GACOSclip/16filt_cum/{}_deramp.png'.format(epoch)
display(png)

In [ ]:
print('effect of removing linear correlation to height:')
print('colour scale: 6pi/cycle')
png='TS_GEOCml2GACOSclip/16filt_cum/{}_hgt_linear.png'.format(epoch)
display(png)

In [ ]:
print('correlation to height')
png='TS_GEOCml2GACOSclip/16filt_cum/{}_hgt_corr.png'.format(epoch)
display(png)

In [ ]:
print('Note: we have avoided spatio-temporal filter using --nofilter option. If run, you would see results here:')
print('effect of spatio-temporal filter:')
png='TS_GEOCml2GACOSclip/16filt_cum/{}_filt.png'.format(epoch)
display(png)

# Viewing the final time series

**cells \[ 22 \]**

The original LiCSBAS script offers advanced visualisation of various output parameters, allows for re-referencing the data etc.

In [ ]:
! LiCSBAS_plot_ts.py -h
# e.g. LiCSBAS_plot_ts.py -i TS_GEOCml2GACOSclip/cum_filt.h5

<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/sample_vel.png" width="550"> <img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/sample_ts.png" width="500">

In [ ]:
print('see the static example of the viewer')
%run ../bin/LiCSBAS_plot_ts.py -i TS_GEOCml2GACOSclip/cum.h5 -p 53/39
# %run ../bin/LiCSBAS_plot_ts.py -i TS_GEOCml2GACOSclip/cum_filt.h5 -p 53/39

In [ ]:
print('use the viewer to see filtered (deramped) dataset - note the time series lines')
%run ../bin/LiCSBAS_plot_ts.py -i TS_GEOCml2GACOSclip/cum_filt.h5 -p 53/39

In [ ]:
print('generating png file of time series for given coordinates')
#! LiCSBAS_plot_ts.py -i TS_GEOCml2GACOSclip/cum.h5 --ts_png test_ts.png -p 53/39 >/dev/null
! LiCSBAS_plot_ts.py -i TS_GEOCml2GACOSclip/cum_filt.h5 --ts_png test_ts.png -p 53/39 >/dev/null
display('test_ts.png')

It is, however, convenient to transform the output data to a standard NetCDF in geographic coordinates, so we can use any standard program for further processing. Following optional scripts will export the results to a standard GeoTIFF file or a NetCDF datacube that you may use both in any GIS environment, or e.g. to load to python via xarray etc.

For more information (and example on QGIS Time Series viewer) see [LiCSBAS wiki](https://github.com/yumorishita/LiCSBAS/wiki/4_other_tools) 

In [ ]:
# to export velocities to a geotiff
! LiCSBAS_flt2geotiff.py -h
# e.g. LiCSBAS_flt2geotiff.py -i TS_GEOCml2*/results/vel -p GEOCml2GACOSclip/EQA.dem_par -o 022D_04826_121209.vel.geo.tif

# to export to Google Earth kmz tiles:
#! LiCSBAS_color_geotiff2tiles.py -h

In [ ]:
# to export all relevant data to a NetCDF datacube
! LiCSBAS_out2nc.py -h
# e.g. LiCSBAS_out2nc.py -i TS_GEOCml2*/cum.h5 -o 022D_04826_121209.nc


Let's export LiCSBAS output to the standard NetCDF datacube, and then view it through jupyter notebook. You may also download the NetCDF file and load into QGIS, ncview etc.

In [ ]:
! LiCSBAS_out2nc.py -i TS_GEOCml2*/cum_filt.h5 -o 022D_04826_121209.nc

In [ ]:
# re-define in case we run this at the end without finalising the processing
import os
frame = '022D_04826_121209'
framedir = os.path.join(os.environ['HOME'],frame)
os.chdir(framedir)

# loading the netcdf as xarray dataset
import xarray as xr
ncfile='022D_04826_121209.nc'
if not os.path.exists(ncfile):
    os.system('LiCSBAS_out2nc.py -i TS_GEOCml2GACOSclip/cum_filt.h5 -o 022D_04826_121209.nc')

cube=xr.open_dataset(ncfile)

# explore the contents
cube

Finally we can visualise the results using an original approach of making the great [pyGMT](https://www.generic-mapping-tools.org/egu22pygmt/intro.html) interactive - with great appreciation to Miss Rochelle Pun, an outstanding student that was selected for the [COMET Summer MSc Internship 2024](https://comet.nerc.ac.uk/internships) during which she developed her [IntPyGmt tool](https://github.com/chelle0425/IntPyGMT) as a side activity.

In [ ]:
# visualise time series
# (c) 2024, Rochelle Pun during her short COMET Summer MSc Internship 2024
# originally published in https://github.com/chelle0425/IntPyGMT
# adapted to LiCSBAS

from LiCSBAS_extras import pygmt_plot_interactive

title = 'LiCSBAS Tutorial: time series'
lims = [-15,60]
label='velocity [mm/year]'
cmap='polar'

# note this will work only with ipympl installed:
%matplotlib widget

pygmt_plot_interactive(cube, title, label, lims, cmap,
                           photobg=False, plotvec=None)

## Citations

Morishita, Y.; Lazecky, M.; Wright, T.J.; Weiss, J.R.; Elliott, J.R.; Hooper, A. LiCSBAS: An Open-Source InSAR Time Series Analysis Package Integrated with the LiCSAR Automated Sentinel-1 InSAR Processor. *Remote Sens.* **2020**, *12*, 424, https://doi.org/10.3390/RS12030424.

Morishita, Y.: Nationwide urban ground deformation monitoring in Japan using Sentinel-1 LiCSAR products and LiCSBAS. *Prog. Earth Planet. Sci.* **2021**, *8*, 6,  https://doi.org/10.1186/s40645-020-00402-7.

Lazecký, M.; Spaans, K.; González, P.J.; Maghsoudi, Y.; Morishita, Y.; Albino, F.; Elliott, J.; Greenall, N.; Hatton, E.; Hooper, A.; Juncu, D.; McDougall, A.; Walters, R.J.; Watson, C.S.; Weiss, J.R.; Wright, T.J. LiCSAR: An Automatic InSAR Tool for Measuring and Monitoring Tectonic and Volcanic Activity. *Remote Sens.* **2020**, *12*, 2430, https://doi.org/10.3390/rs12152430.

Lazecký, M.; Ou, Q.; McGrath, J.; Payne, J.; Espin, P.; Hooper, A.; Wright, T. Strategies for improving and correcting unwrapped interferograms implemented in LiCSBAS. *Procedia Computer Science* **2024**, *239*, 2408-2412, https://doi.org/10.1016/j.procs.2024.06.435.

## Acknowledgements

The core and main functionality of LiCSBAS has been accomplished by Dr. Yu Morishita (Geospatial Information Authority of Japan) during his visit at the University of Leeds, funded by JSPS Overseas Research Fellowship (June 2018-March 2020). LiCSBAS is further developed by the team at the University of Leeds, under the supervision of Prof. Andy Hooper and Prof. Tim Wright.  

COMET is the UK Natural Environment Research Council's Centre for the Observation and Modelling of Earthquakes, Volcanoes and Tectonics. LiCSAR is developed as part of the NERC large grant, "Looking inside the continents from Space" (NE/K010867/1). LiCSAR contains modified Copernicus Sentinel-1 data analysed by the COMET. LiCSAR uses [JASMIN](http://jasmin.ac.uk), the UK’s collaborative data analysis environment.  

The [Scientific Colour Maps](http://www.fabiocrameri.ch/colourmaps.php) ([Crameri, 2018](https://doi.org/10.5194/gmd-11-2541-2018)) is used in LiCSBAS.


[<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/COMET_logo.png"  width="100">](https://comet.nerc.ac.uk/)   [<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/logo-leeds.png"  width="250">](https://environment.leeds.ac.uk/see/)  [<img src="https://raw.githubusercontent.com/wiki/yumorishita/LiCSBAS/images/LiCS_logo.jpg"  width="150">](https://comet.nerc.ac.uk/COMET-LiCS-portal/)  [<img src="https://mybinder.org/static/logo.svg" width="200">](https://mybinder.org/)